## Importing

In [1]:
import requests
import json
import pandas as pd
import keyring
from datetime import datetime
from datetime import timedelta

## Bring In Data

In [2]:
# defining credentials for auth
user = 'wramsey'
pwd = keyring.get_password("snow_prod", "wramsey")

# List of "sysparm_fields" being pulled
# u_repeat, upon_reject, sys_updated_on, number, u_vcc, state, sys_created_by, knowledge, u_external_customer_impacted, u_ach, cmdb_ci, impact, active, priority, sys_domain_path, business_duration, u_avidpay_check, u_core, u_ipay, u_avidpay_direct, short_description, work_start, u_check, notify, sys_class_name, closed_by, parent_incident, u_choice_6, reopened_by, u_utility, reassignment_count, assigned_to, u_board, escalation, upon_approval, u_payments_delayed, made_sla, u_money_movement, child_incidents, task_effective_number, resolved_by, sys_updated_by, opened_by, sys_created_on, sys_domain, u_dollar_impact, calendar_stc, closed_at, business_service, opened_at, caller_id, reopened_time, resolved_at, u_hub, subcategory, close_code, assignment_group, u_pir_requested, business_stc, calendar_duration, close_notes, sys_id, contact_type, incident_state, urgency, severity, approval, sys_mod_count, reopen_count, location, category

# Set the request parameters - assigned to support center in last 3 months
url = 'https://avidxchange.service-now.com/api/now/table/incident?sysparm_query=assignment_group%3Dfe169ea0db5e2700bd3ec170ba961941%5Eopened_at%3Ejavascript%3Ags.beginningOfLast3Months()&sysparm_fields=u_repeat%2C%20upon_reject%2C%20sys_updated_on%2C%20number%2C%20u_vcc%2C%20state%2C%20sys_created_by%2C%20knowledge%2C%20u_external_customer_impacted%2C%20u_ach%2C%20cmdb_ci%2C%20impact%2C%20active%2C%20priority%2C%20sys_domain_path%2C%20business_duration%2C%20u_avidpay_check%2C%20u_core%2C%20u_ipay%2C%20u_avidpay_direct%2C%20short_description%2C%20work_start%2C%20u_check%2C%20notify%2C%20sys_class_name%2C%20closed_by%2C%20parent_incident%2C%20u_choice_6%2C%20reopened_by%2C%20u_utility%2C%20reassignment_count%2C%20assigned_to%2C%20u_board%2C%20escalation%2C%20upon_approval%2C%20u_payments_delayed%2C%20made_sla%2C%20u_money_movement%2C%20child_incidents%2C%20task_effective_number%2C%20resolved_by%2C%20sys_updated_by%2C%20opened_by%2C%20sys_created_on%2C%20sys_domain%2C%20u_dollar_impact%2C%20calendar_stc%2C%20closed_at%2C%20business_service%2C%20opened_at%2C%20caller_id%2C%20reopened_time%2C%20resolved_at%2C%20u_hub%2C%20subcategory%2C%20close_code%2C%20assignment_group%2C%20u_pir_requested%2C%20business_stc%2C%20calendar_duration%2C%20close_notes%2C%20sys_id%2C%20contact_type%2C%20incident_state%2C%20urgency%2C%20severity%2C%20approval%2C%20sys_mod_count%2C%20reopen_count%2C%20location%2C%20category&sysparm_limit=1000'

# Set proper headers
headers = {"Content-Type":"application/json","Accept":"application/json"}

# Do the HTTP request
response = requests.get(url, auth=(user, pwd), headers=headers )

# Check for HTTP codes other than 200
if response.status_code != 200:
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
    print('Status:', response.status_code, '\n')
    print('Headers:', response.headers, '\n')
    data = response.json()
    print(json.dumps(data, indent=2))

# Decode the JSON response into a dictionary and use the data
data = response.json()

# this turns the dictionary into a pandas df
df_main = pd.DataFrame.from_dict(data['result'])
pd.set_option('max_columns', None)

# Write out to a CSV file
df_main.to_csv('C:\\Users\\wramsey\\Desktop\\snow_last_60.csv', index=False)

## Printing INC before time-shift is applied

In [3]:
# Create dataframe showing 3 INC
df_temp = df_main[ (df_main['task_effective_number'] == 'INC0065154') |
         (df_main['task_effective_number'] == 'INC0066309') |
         (df_main['task_effective_number'] == 'INC0070921') |
         (df_main['task_effective_number'] == 'INC0069720')]

# write out the INC named
df_temp[['sys_created_on', 'opened_at', 'task_effective_number', 'sys_created_by']]

,sys_created_on,opened_at,task_effective_number,sys_created_by
0,2021-03-11 13:05:17,2021-03-11 13:05:16,INC0066309,aoshields
1,2021-04-26 15:55:27,2021-04-26 15:54:46,INC0070921,X110405
20,2021-04-16 13:13:05,2021-04-16 13:12:37,INC0069720,X110405


## After time-shift applied

In [4]:
# With Time-Shift applied
# write time-columns to list
time_columns = ['sys_updated_on', 'sys_created_on', 'closed_at', 'opened_at', 'business_duration', 'reopened_time',
                'resolved_at', 'work_start', 'calendar_duration']

# for each column, apply to_datetime function
for item in time_columns:
    df_main[item] = pd.to_datetime(df_main[item])

# Subtract 4 hours from the time column to match what is show in ServiceNow
df_main['opened_at'] = df_main['opened_at'] - timedelta(hours=4)
df_main['sys_created_on'] = df_main['sys_created_on'] - timedelta(hours=4)

# Create dataframe showing 3 INC with time-shift applied
df_temp = df_main[ (df_main['task_effective_number'] == 'INC0065154') |
         (df_main['task_effective_number'] == 'INC0066309') |
         (df_main['task_effective_number'] == 'INC0070921') |
         (df_main['task_effective_number'] == 'INC0069720')]

# write out the INC named
df_temp[['sys_created_on', 'opened_at', 'task_effective_number', 'sys_created_by']]

,sys_created_on,opened_at,task_effective_number,sys_created_by
0,2021-03-11 09:05:17,2021-03-11 09:05:16,INC0066309,aoshields
1,2021-04-26 11:55:27,2021-04-26 11:54:46,INC0070921,X110405
20,2021-04-16 09:13:05,2021-04-16 09:12:37,INC0069720,X110405


## Whole dataframe (with all columns included)

In [5]:
# Whole dataframe (with all columns included)
df_temp

,u_repeat,made_sla,upon_reject,u_money_movement,sys_updated_on,child_incidents,task_effective_number,number,u_vcc,resolved_by,sys_updated_by,opened_by,sys_created_on,sys_domain,u_dollar_impact,state,sys_created_by,knowledge,u_external_customer_impacted,u_ach,calendar_stc,closed_at,cmdb_ci,impact,active,business_service,priority,sys_domain_path,opened_at,business_duration,caller_id,reopened_time,resolved_at,u_hub,u_avidpay_check,u_core,u_ipay,subcategory,u_avidpay_direct,short_description,work_start,close_code,assignment_group,u_pir_requested,u_check,business_stc,calendar_duration,notify,sys_class_name,close_notes,closed_by,parent_incident,u_choice_6,sys_id,contact_type,reopened_by,incident_state,urgency,u_utility,reassignment_count,assigned_to,severity,approval,sys_mod_count,reopen_count,u_board,escalation,upon_approval,u_payments_delayed,location,category
0,false,true,cancel,false,2021-03-19 15:00:01,0,INC0066309,INC0066309,false,{'link': 'https://avidxchange.service-now.com/...,system,{'link': 'https://avidxchange.service-now.com/...,2021-03-11 09:05:17,{'link': 'https://avidxchange.service-now.com/...,0,7,aoshields,false,false,false,7067,2021-03-19 15:00:01,{'link': 'https://avidxchange.service-now.com/...,3,false,{'link': 'https://avidxchange.service-now.com/...,3,/,2021-03-11 09:05:16,1970-01-01 01:57:47,{'link': 'https://avidxchange.service-now.com/...,NaT,2021-03-11 15:03:03,false,false,false,false,Salesforce (Org1 - Sales),false,Can not log in to salesforce.,NaT,Closed/Resolved by Caller,{'link': 'https://avidxchange.service-now.com/...,false,false,7067,1970-01-09 01:54:45,1,incident,Resolution Notes (Sent to Customer) copied fro...,{'link': 'https://avidxchange.service-now.com/...,{'link': 'https://avidxchange.service-now.com/...,1,0001228adbb660d09f9aec51ca961920,Self Service,,7,1,false,0,{'link': 'https://avidxchange.service-now.com/...,3,not requested,16,0,false,0,proceed,false,{'link': 'https://avidxchange.service-now.com/...,software
1,false,true,cancel,false,2021-04-26 15:56:05,0,INC0070921,INC0070921,false,{'link': 'https://avidxchange.service-now.com/...,amacasaet,{'link': 'https://avidxchange.service-now.com/...,2021-04-26 11:55:27,{'link': 'https://avidxchange.service-now.com/...,0,6,X110405,false,false,false,41,NaT,{'link': 'https://avidxchange.service-now.com/...,3,true,{'link': 'https://avidxchange.service-now.com/...,4,/,2021-04-26 11:54:46,1970-01-01 00:00:41,{'link': 'https://avidxchange.service-now.com/...,NaT,2021-04-26 15:55:27,false,false,false,false,,false,"Hi, I'm new. I think I need a monitor cable.",NaT,Solved (Work Around),{'link': 'https://avidxchange.service-now.com/...,false,false,41,NaT,1,incident,user given wrong monitor cable. setting mini t...,,,1,0011ccc9db3f28105cfba895ca961942,Chat,,6,2,false,0,{'link': 'https://avidxchange.service-now.com/...,3,not requested,23,0,false,0,proceed,false,,
20,false,true,cancel,false,2021-04-23 14:00:00,0,INC0069720,INC0069720,false,{'link': 'https://avidxchange.service-now.com/...,system,{'link': 'https://avidxchange.service-now.com/...,2021-04-16 09:13:05,{'link': 'https://avidxchange.service-now.com/...,0,7,X110405,false,false,false,839,2021-04-23 14:00:00,{'link': 'https://avidxchange.service-now.com/...,3,false,{'link': 'https://avidxchange.service-now.com/...,4,/,2021-04-16 09:12:37,1970-01-01 00:13:59,{'link': 'https://avidxchange.service-now.com/...,NaT,2021-04-16 13:26:36,false,false,false,false,,false,authenticator,NaT,Solved (Permanently),{'link': 'https://avidxchange.service-now.com/...,false,false,839,1970-01-08 00:47:23,1,incident,reset for user in AAD,{'link': 'https://avidxchange.service-now.com/...,,1,030c73e1db2364d020acde82ca96199e,Chat,,7,2,false,0,{'link': 'https://avidxchange.service-now.com/...,3,not requested,9,0,false,0,proceed,false,,System Access
